In [1]:
import openai
import re

import pandas as pd

from langchain import PromptTemplate
from langchain.llms import OpenAI

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

import pickle
import datetime
from river import metrics, optim, reco

- 好きなアニメを取得したい


In [2]:
def get_prompt(template, input_text):
    prompt = PromptTemplate(template=template, input_variables=["input"])
    prompt_text = prompt.format(input=input_text)

    return prompt_text


def get_anime_name(anime_name):
    anime_list = anime_name.split("\n")
    anime_list = [anime for anime in anime_list if anime]

    return anime_list


def create_index_data():
    df = pd.read_csv("../data/anime_list.csv")
    df = df[["MAL_ID", "Name", "English name", "Japanese name", "Genres"]]

    # lowr
    df["Name"] = df["Name"].str.lower()
    df["English name"] = df["English name"].str.lower()
    df["Japanese name"] = df["Japanese name"].str.lower()
    df["Genres"] = df["Genres"].str.lower()

    df.rename(columns={"MAL_ID": "anime_id"}, inplace=True)
    df.to_csv("../data/index_data.csv", index=False)


def create_db():
    loader = CSVLoader(file_path="../data/index_data.csv")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(docs, embeddings)
    db.save_local("../data/faiss_index")


def check_similarity_item(db, input):
    input = input.lower()
    sim = db.similarity_search_with_score(query=input, k=1)

    anime_id = sim[0][0].page_content.split("\n")[0]
    match = re.search(r"anime_id:\s*(\d+)", anime_id)
    if match:
        anime_id = int(match.group(1))
    else:
        anime_id = np.nan

    anime_name = sim[0][0].page_content.split("\n")[1]
    sim_value = sim[0][1]

    print(anime_id, anime_name, sim_value)


def get_similarity_item(db, input):
    input = input.lower()
    sim = db.similarity_search_with_score(query=input, k=1)

    anime_id = sim[0][0].page_content.split("\n")[0]
    match = re.search(r"anime_id:\s*(\d+)", anime_id)
    if match:
        anime_id = int(match.group(1))
    else:
        anime_id = np.nan

    anime_name = sim[0][0].page_content.split("\n")[1]
    sim_value = sim[0][1]

    if sim_value >= 0.3:
        return None
    else:
        return anime_id

### Create DB


In [3]:
create_index_data()

In [4]:
# create_db()

In [5]:
embeddings = OpenAIEmbeddings()
db = FAISS.load_local("../data/faiss_index", embeddings)

In [6]:
llm = OpenAI()

In [7]:
check_anime_list = [
    "Angel Beats",
    "魔法使いプリキュア",
    "Mahoutsukai Precure!",
    "狼と香辛料",
    "Ookami to Koushinryo",
    "鬼滅の刃",
    "Kimetsu no Yaiba",
]

In [8]:
for anime in check_anime_list:
    check_similarity_item(db, anime)

6547 Name: angel beats! 0.25639322
31884 Name: mahoutsukai precure! 0.31024
31884 Name: mahoutsukai precure! 0.20283352
2966 Name: ookami to koushinryou 0.34811774
6007 Name: ookami to koushinryou ii: ookami to kohakuiro no yuuutsu 0.26174277
30679 Name: queen's blade: grimoire 0.33600473
38000 Name: kimetsu no yaiba 0.1943137


### Prompt


In [9]:
# user_input = """おすすめのアニメを教えてください。私が好きなアニメは以下の通りです。

# Angel Beats!
# Mahoutsukai Precure!
# Ookami to Koushinryou
# """

user_input = """おすすめのアニメを教えてください。私が好きなアニメは以下の通りです。

Angel Beats!
Mahoutsukai Precure!
Ookami to Koushinryou
Shinsekai yori
Shugo Chara!
Uma Musume: Pretty Derby (TV)
Fate/Zero
Uchuu Senkan Yamato 2199
"""

In [10]:
template_user_input = """以下の文章は、ユーザが好きなアニメの名前を列挙しています。
アニメの名前を抽出し、出力しなさい。

{input}
"""

In [11]:
user_prompt = get_prompt(template_user_input, user_input)

In [12]:
candidate_list = llm(user_prompt)

In [13]:
candidate_list = get_anime_name(candidate_list)

In [14]:
candidate_list

['Angel Beats!',
 'Mahoutsukai Precure!',
 'Ookami to Koushinryou',
 'Shinsekai yori',
 'Shugo Chara!',
 'Uma Musume: Pretty Derby (TV)',
 'Fate/Zero',
 'Uchuu Senkan Yamato 2199']

In [15]:
rated_anime_ids = []
for item in candidate_list:
    rated_anime_ids.append(get_similarity_item(db, item))

In [16]:
rated_anime_ids

[6547, 31884, 6007, 13125, 2923, 35249, 10087, 12029]

In [17]:
# anime_ids = [6547, 31884, 6007]

In [18]:
anime_list = pd.read_csv("../data/index_data.csv")

In [19]:
anime_list[anime_list["anime_id"].isin(rated_anime_ids)]

,anime_id,Name,English name,Japanese name,Genres
1742,2923,shugo chara!,shugo chara!,しゅごキャラ！,"comedy, magic, school, shoujo"
2467,6007,ookami to koushinryou ii: ookami to kohakuiro ...,unknown,狼と香辛料ii 狼と琥珀色の憂鬱,"adventure, fantasy, historical, romance"
2565,6547,angel beats!,angel beats!,angel beats!（エンジェルビーツ）,"action, comedy, drama, school, supernatural"
2945,10087,fate/zero,fate/zero,フェイト/ゼロ,"action, supernatural, magic, fantasy"
3140,12029,uchuu senkan yamato 2199,star blazers:space battleship yamato 2199,宇宙戦艦ヤマト2199,"action, military, sci-fi, space, drama"
3210,13125,shinsekai yori,from the new world,新世界より,"drama, horror, mystery, psychological, sci-fi,..."
4144,31884,mahoutsukai precure!,maho girls precure!,魔法つかいプリキュア！,"action, slice of life, magic, fantasy, school,..."
4550,35249,uma musume: pretty derby (tv),umamusume:pretty derby,ウマ娘 プリティーダービー,"comedy, slice of life, sports"


In [20]:
# online learning

In [21]:
with open("../data/model.pkl", "rb") as f:
    model = pickle.load(f)

In [22]:
def create_user_uid():
    now = datetime.datetime.now()
    user_id = now.strftime("%Y%m%d%H%M%S")
    user_id = int(user_id)

    return user_id

In [23]:
user_id = create_user_uid()

In [24]:
dataset = anime_list[anime_list["anime_id"].isin(rated_anime_ids)]

In [25]:
dataset = dataset[dataset["anime_id"].isin(rated_anime_ids)]
dataset["user_id"] = user_id
dataset["rating"] = 10

In [26]:
dataset = dataset[["user_id", "anime_id", "rating"]].to_dict(orient="records")

In [27]:
metric = metrics.MAE() + metrics.RMSE()

In [28]:
# Update model
for data in dataset:
    y_pred = model.predict_one(user=data["user_id"], item=data["anime_id"])
    metric = metric.update(data["rating"], y_pred)
    model = model.learn_one(
        user=data["user_id"], item=data["anime_id"], y=data["rating"]
    )

In [29]:
metric

MAE: 1.822499, RMSE: 1.865068

In [30]:
def predict(user_id, anime_list, rated_anime_ids):
    # predict recommend item
    result_df = model.rank(user=user_id, items=anime_list["anime_id"])
    result_df = pd.DataFrame(result_df, columns=["anime_id"])
    result_df = pd.merge(result_df, anime_list, on=["anime_id"], how="inner")

    # # remove rated item
    result_df = result_df[~result_df["anime_id"].isin(rated_anime_ids)]
    result_df = result_df.reset_index(drop=True)[0:10]

    return result_df

In [31]:
recommend_result = predict(user_id, anime_list, rated_anime_ids)

In [32]:
recommend_result

,anime_id,Name,English name,Japanese name,Genres
0,2004,hanada shounen-shi,unknown,花田少年史,"comedy, drama, seinen, slice of life, supernat..."
1,457,mushishi,mushi-shi,蟲師,"adventure, slice of life, mystery, historical,..."
2,9253,steins;gate,steins;gate,steins;gate,"thriller, sci-fi"
3,820,ginga eiyuu densetsu,legend of the galactic heroes,銀河英雄伝説,"military, sci-fi, space, drama"
4,12431,uchuu kyoudai,space brothers,宇宙兄弟,"comedy, sci-fi, seinen, slice of life, space"
5,5941,cross game,cross game,クロスゲーム,"comedy, drama, romance, school, shounen, sports"
6,31098,ushio to tora (tv) 2nd season,ushio & tora,うしおととら,"action, adventure, comedy, demons, supernatura..."
7,33095,shouwa genroku rakugo shinjuu: sukeroku futata...,descending stories:showa genroku rakugo shinju,昭和元禄落語心中～助六再び篇～,"drama, historical, josei"
8,21939,mushishi zoku shou,mushi-shi -next passage-,蟲師 続章,"adventure, slice of life, mystery, historical,..."
9,11061,hunter x hunter (2011),hunter x hunter,hunter×hunter（ハンター×ハンター）,"action, adventure, fantasy, shounen, super power"


In [33]:
chain = RetrievalQA.from_chain_type(
    # llm, retriever=db.as_retriever(search_kwargs={"k": 1})
    llm,
    retriever=db.as_retriever(),
)

In [34]:
template_recommend = f"""あなたはおすすめのアニメをレコメンドするレコメンドシステムです。
以下の情報をまとめて、ユーザにアニメを紹介してください。
また、そのアニメのジャンルも教えてください。ジャンルは日本語で表示してください。

--
アニメ名: {recommend_result["Name"].values[0]}
アニメ名: {recommend_result["Name"].values[1]}
アニメ名: {recommend_result["Name"].values[2]}
"""

In [35]:
template_recommend = f"""あなたはおすすめのアニメをレコメンドするレコメンドシステムです。
以下の情報をまとめて、ユーザにアニメを紹介してください。
また、そのアニメのジャンルも教えてください。アニメ名、ジャンルは日本語で表示してください。

----
おすすめアニメリスト:
アニメ名: {recommend_result["Name"].values[0]}
アニメ名: {recommend_result["Name"].values[1]}
アニメ名: {recommend_result["Name"].values[2]}
アニメ名: {recommend_result["Name"].values[3]}
アニメ名: {recommend_result["Name"].values[4]}
----

出力は以下の形式で出力してください。
----
出力例:
以下がおすすめのアニメです。
1. アニメ名 - ジャンルはX, Y, Zです。
2. アニメ名 - ジャンルはX, Y, Zです。
3. アニメ名 - ジャンルはX, Y, Zです。
4. アニメ名 - ジャンルはX, Y, Zです。
5. アニメ名 - ジャンルはX, Y, Zです。
----
"""

In [36]:
result = chain(template_recommend)

In [37]:
print(result["result"])

 以下がおすすめのアニメです。
1. ハナダ少年史 - ジャンルは短編劇場、ドラマ、青春です。
2. 蟲師 - ジャンルは冒険、ファンタジー、スリラーです。
3. シュタインズ・ゲート - ジャンルはサスペンス、科学フィクションです。
4. 銀河英雄伝説 - ジャンルは冒険、SF、ファンタジーです。
5. 宇宙兄弟 - ジャンルは冒険、コメディ、宇宙、SFです。


In [38]:
recommend_result[["Name", "Japanese name", "Genres"]].head(5)

,Name,Japanese name,Genres
0,hanada shounen-shi,花田少年史,"comedy, drama, seinen, slice of life, supernat..."
1,mushishi,蟲師,"adventure, slice of life, mystery, historical,..."
2,steins;gate,steins;gate,"thriller, sci-fi"
3,ginga eiyuu densetsu,銀河英雄伝説,"military, sci-fi, space, drama"
4,uchuu kyoudai,宇宙兄弟,"comedy, sci-fi, seinen, slice of life, space"
